In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# family = "AppleGothic"
family = "Malgun Gothic"

plt.rc('font', family=family)

In [ ]:
df = pd.read_csv('data/서울특별시/apartment_df.csv', index_col=0)
df

In [ ]:
corr_df = df.corr()
corr_df

In [ ]:
sns.heatmap(corr_df, annot=True, cmap="RdYlBu_r", vmin=-1, vmax=1)

In [ ]:
sns.clustermap(corr_df, annot=True, cmap="RdYlBu_r", vmin=-1, vmax=1)